In [ ]:
from scipy import stats
from multiprocessing import Pool
from statsmodels.stats.multitest import fdrcorrection

os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/new_count/")

def preprocess_df():
    ####################################################
    result_dir = "sig/binomi/"
    os.system("mkdir -p %s" % result_dir)
    result_file = os.path.join(result_dir, "Fus2.txt")          ####################
    ####################################################
    input_file = "Fus2-inp_L4_710D01.readcounts.txt" ####################
    df = pd.read_table(input_file)
    #####################################################
    df = df[(df['refCount']>=2) & (df['altCount']>=2)]
    df = df[(df['refCount'] + df['altCount'])>=10]
    #######################################################
    df['allelicRatio'] = df['refCount'] / (df['refCount'] + df['altCount'])
    pvalue_list = []
    for a, b in df.iterrows():
        ref, alt = b['refCount'], b['altCount']
        total = ref + alt
        pvalue = stats.binom_test(alt, n=total, p=0.5)
        pvalue_list.append(pvalue)
    qvalue_list = list(fdrcorrection(pvalue_list)[1])
    df['bino_pvalue'] = pvalue_list
    df['qvalue'] = qvalue_list
    df.to_csv(result_file, sep="\t", index=False)
###
preprocess_df()